
$\qquad$ $\qquad$$\qquad$  **TDA206/DIT206 Discrete Optimization: Home Assignment 2 -- Integer LP and Relaxation** <br />
$\qquad$ $\qquad$$\qquad$                   **Grader: Adam Breitholtz** <br />
$\qquad$ $\qquad$$\qquad$                   **Due Date: 12th Feb** <br />
$\qquad$ $\qquad$$\qquad$                   **Submitted by: Mirco Ghadri, 010421-1693, mircog@chalmers.se** <br />


---


General guidelines:
*   All solutions to theoretical and pratical problems must be submitted in this ipynb notebook, and equations wherever required, should be formatted using LaTeX math-mode.
*   All discussion regarding practical problems, along with solutions and plots should be specified in this notebook. All plots/results should be visible such that the notebook do not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.
*   Your name, personal number and email address should be specified above.
*   All tables and other additional information should be included in this notebook.
*   Before submitting, make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.


# Question 1.
(5 points) In the previous assignment, we solved the transportation problem of the space colonies using LP. Explain why the solution was integral with a proof.

In [2]:
import numpy as np
import cvxpy as cp

For reference, here is the transportation problem:

\begin{array}{l|c|c|c|c|c} 
      & Triacus & New Berlin  & Strnad  & Vega  & supply\\ \hline
 Farpoint &   6 &  9 & 10 & 8 & 35\\
 Yorktown &  9 & 5 & 16 & 14 & 40\\
 Earhart & 12 &  7 & 13 & 9 & 50\\ \hline
    demand & 20 &30&30&45& \left(\sum=125\right) \\ 
\end{array}

In order to understand why the solution was integral, we need to look at the constraint matrix and the constraint vector. In order for an LP to be integral, the constraint matrix needs to be a TU(Totally Unimodular) matrix and the constraint vector need to be integral. We have 2 constraint vectors, s and d. Both s and d are integral so that condition is satisfied. Our constraint matrix, lets call it $M$, is what we multiply by x. In the case of our supply constraints, the inequality becomes $M_{1}x \leq s$. In the case of our demand constraints, we use another constraint Matrix, lets call it $M_{2}$. Then our inequality becomes $M_{2}x \geq d$. We need to prove that $M_{1}$ and $M_{2}$ are both TU matrices.

In [6]:
A = np.array([[6,9,10,8],[9,5,16,14],[12,7,13,9]])
s = np.array([35,40,50])
d = np.array([20,30,30,45])
X = cp.Variable(shape = A.shape, nonneg=True)
obj = cp.Minimize(cp.sum(cp.multiply(A,X)))
constraints = [cp.sum(X,axis=1) <= s, cp.sum(X,axis=0) >= d]
problem = cp.Problem(obj,constraints)
result = problem.solve()

In [7]:
result

1020.0000006004018

When looking at the formulation of the LP from assignment 1, we see that our constraints are not written on the form
$$
Ax \leq b \
Ax \geq b
$$
This needs to be changed. But this requires us to reformulate a lot of the code as well.

In [14]:
A = np.array([6,9,10,8, 9,5,16,14, 12,7,13,9])
s = np.array([35,40,50])
d = np.array([20,30,30,45])
X = cp.Variable(shape = A.shape, nonneg=True)
obj = cp.Minimize(A @ X)
M_1 =  np.array(
        [[1,1,1,1, 0,0,0,0, 0,0,0,0],
        [0,0,0,0, 1,1,1,1, 0,0,0,0],
        [0,0,0,0, 0,0,0,0, 1,1,1,1]]
)
M_2 = np.array(
       [[1,0,0,0, 1,0,0,0, 1,0,0,0],
       [0,1,0,0, 0,1,0,0, 0,1,0,0],
       [0,0,1,0, 0,0,1,0, 0,0,1,0],
       [0,0,0,1, 0,0,0,1, 0,0,0,1]]
)
constraints = [M_1 @ X <= s, M_2 @ X >= d]
problem = cp.Problem(obj,constraints)
result = problem.solve()

In [15]:
result

1020.0000006004018

We can see that the 2 formulations of the code have the same solution, i.e. 1020. We now claim that both $M_{1}$ and $M_{2}$ are TU matrices.
$$
M_1 = \begin{pmatrix}
1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 \\
\end{pmatrix}
$$

$$
M_2 = \begin{pmatrix}
1 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 1 \\
\end{pmatrix}
$$

Both $M_{1}$ and $M_{2}$ satisfy the 2 sufficient conditions for a TU matrix as well as the prerequisite.  
**Prerequisite:** All values in the matrices are either 0,-1 or 1. ✅  
**Sufficient conditions:**  
1. Every column has at most 2 non-zero entries. ✅
2. There exists a partition of the rows in the matrix into 2 sets $R_{1}$ and $R_{2}$ such that the sum of the values in row $R_{1}$ equals the sum of the values in row $R_{2}$ for each column $C_{i}$ with 2 non-zero entries. (This is automatically satisfied since we have no columns with 2 non-zero entries). ✅

# Question 2.

Recall the Minimum Weight Vertex Cover (VC) Problem: Given an undirected graph $G=(V, E)$, with node set $V$ and edge set $E$, where each node has a positive weight $w(v)$ associated with it (see figure), the goal is to select a subset $V'\subseteq V$ of nodes such that every edge has at least one node incident to it, and the total selected node weight $\sum_{v\in V'} w(v)$ is minimized. 

* (4 points) Formulate the ILP for the VC problem for the example below, and solve it using **CVXPY** integer solver, for instance, `myVar = cp.Variable(<dim>, integer=True)`.
* (2 points) Pass to the LP relaxation and solve it using **CVXPY** and comment on the relation between the two solutions.
* (2 points) Apply the rounding rule discussed in class to the optimal LP solution to obtain a solution to the ILP and compare it to the optimal ILP solution.

<img src="vertex_cover.png" alt="alt text" width="200"/>


**a)**

In [16]:
#weights of each vertex
W = np.array([4,2,3,2,1,4])
#Whether or not vertex x is in the vertex cover
X = cp.Variable(6, integer=True) #nonneg=True
#constraint vector
ones = np.array([1,1,1,1,1,1,1])
#edge matrix: edges*vertices. For each edge, show which vertices cover that edge. Vertices are in the same order as they occur in the weight matrix.
#edges occur in the order from the top-most edge to the bottom-most edge.
e = np.array([[1,1,0,0,0,0],
             [1,0,0,1,0,0],
             [1,0,1,0,0,0],
             [0,1,0,1,0,0],
             [0,0,1,1,0,0],
             [0,0,1,0,1,0],
             [0,0,0,1,0,1]])

#adjacency matrix. For each vertex, shows which vertex it is adjacent to
adj_matrix = np.array([[0,1,1,1,0,0],
                       [1,0,0,1,0,0],
                       [1,0,0,1,1,0],
                       [1,1,1,0,0,1],
                       [0,0,1,0,0,0],
                       [0,0,0,0,1,0]])

constraints = []
for i in range(6):
    constraints.append(X[i]>=0)

#1. solution using adjacency matrix
# for u in range(len(adj_matrix)):
#     for v in range(len(adj_matrix[0])):
#         if adj_matrix[u, v] == 1:
#             constraints.append(X[u] + X[v] >= 1)

#2. solution using edge matrix
constraints += [cp.matmul(e,X)>=ones]

objective = cp.Minimize(W @ X)
problem = cp.Problem(objective,constraints)
result = problem.solve()

In [17]:
result

7.0

**b)**

In [18]:
#weights of each vertex
W = np.array([4,2,3,2,1,4])
#Whether or not vertex x is in the vertex cover
X = cp.Variable(6, nonneg=True) #nonneg=True
ones = np.array([1,1,1,1,1,1,1])
e = np.array([[1,1,0,0,0,0],
             [1,0,0,1,0,0],
             [1,0,1,0,0,0],
             [0,1,0,1,0,0],
             [0,0,1,1,0,0],
             [0,0,1,0,1,0],
             [0,0,0,1,0,1]])

adj_matrix = np.array([[0,1,1,1,0,0],
                       [1,0,0,1,0,0],
                       [1,0,0,1,1,0],
                       [1,1,1,0,0,1],
                       [0,0,1,0,0,0],
                       [0,0,0,0,1,0]])
constraints = []
# for u in range(len(adj_matrix)):
#     for v in range(len(adj_matrix[0])):
#         if adj_matrix[u, v] == 1:
#             constraints.append(X[u] + X[v] >= 1)
constraints += [cp.matmul(e,X)>=ones]

objective = cp.Minimize(cp.sum(cp.multiply(W,X)))
problem = cp.Problem(objective,constraints)
result = problem.solve()

In [19]:
result

6.999999999408789

In [20]:
X.value

array([0.48469594, 0.51530406, 0.51530406, 1.        , 0.48469594,
       0.        ])

**c)**

The solution when rounding the optimal solution to the LP has the same value as the solution to the ILP. The value is 7. It also happens to be the same value as the solution to the LP without rounding the x coordinates.

In [21]:
np.sum(np.where(X.value < 0.5, 0, 1) * W)

7

# Question 3.

Consider a number of interpreters (Olof, Petra, Qamar,
  Rachel, Soren and Tao), as well as a set of languages (Arab,
  Bengali, Cantonese, Dutch, English, French and German). Each
  interpreter speaks a number of different languages (abbreviated by
  first letter), and has a certain per-diem integer cost:

\begin{array}{lll}
Interpreter & Languages & Cost\\
O & ABD & 3\\
P & C & 1\\
Q & CDG & 1\\
R & B & 2\\
S & G & 4\\
T & EF & 1\\
\end{array}

* (2 points) A *hypergraph* is a structure $H = (V,E)$ where $V$ is a set of vertices and $E$ is a collection of subsets of $V$. The special case when all subsets $e \in E$ have size exactly $2$ corresponds to the familiar case of a graph. A vertex cover in such a hypergraph is a subset $U \subseteq V$ such that $e \cap U \not = \emptyset$ for each $e \in E$ (note that this reduces to the usual vertex cover in graphs). Show that the problem of finding interpreters can be formulated as a vertex cover problem in a suitable hypergraph.
* (4 points) Develop a ILP formulation to finding the vertex cover of minimum cost in a hypergraph. The hypergraph can be represented as a $|V| \times |E|$ binary matrix $A$ where $A[i,j] = 1$ iff vertex $i$ is in edge $j$ and 0 otherwise. The costs for vertices are in an array $\texttt{c}$ where the cost of picking vertex $i$ is $c[i]$. Use the ILP formulation for the VC problem to hire the cheapest set of interpreters such that all languages are covered. Input the data above manuallly and solve it using **CVXPY**'s integer solver.
* (2 points) Pass to the LP relaxation and solve it using **CVXPY**.
* (2 points) Explain why the two solutions above are same (different).


**a)** We can formulate it as a vertex cover problem by considering each interpreter as a vertex. The selected interpreters form a vertex cover if they cover all of the languages, i.e the languages spoken by all of the interpreters combined includes all of the languages from A to F. Hence, the languages are the edges that needs to be covered in this case. However, these edges do not have to have size 2. For example, a language could be covered by 2 vertices or by only 1 vertex. Take the language E and F for example. They are only covered by the vertex T. Hence, the edge would be an edge with only 1 element, which is T. The language B is covered by 2 vertices, O and R, so it is an egde with 2 vertices, i.e. {O,R}.

**b)**

In [22]:
#whether vertex x is in the vertex cover or not
X = cp.Variable(6,integer=True)
#whether edge e is in the vertex cover or not. This method was not used.
E = cp.Variable(6,integer=True)

#costs of each vertex - the vertices are always in the order O,P,Q,R,S,T
c = np.array([3,1,1,2,4,1])

objective = cp.Minimize(cp.sum(cp.multiply(c,X)))

#A*E matrix, The rows represent the vertex. The column represents the edge: A[i][j]==1 if vertex i covers edge j.
#edges = {A:{O},B:{O,R},C:{Q,P},D:{O,Q},E:{T},F:{T},G:{S,Q}} - Edge E and F are merged to one edge
#This method was not used as it was not straight-forward to use.
a = np.array([[1,1,0,1,0,0],[0,0,1,0,0,0],[0,0,1,1,0,1],[0,1,0,0,0,0],[0,0,0,0,0,1],[0,0,0,0,1,0]])

#A*A adjacency matrix: A[i][j]=1 if node i and node j have an edge. An edge is a subset of vertices that speak a certain language.
#edges = {A:{O},B:{O,R},C:{Q,P},D:{O,Q},E:{T},F:{T},G:{S,Q}} - Edge E and F are merged to one edge
#Adjacency matrix might be the more complicated representation of this graph. Therefore we provide an explanation for the rows.
#[1,0,1,0,0,0]: Vertex O has an edge with itself and with vertex R and with vertex Q
#[0,0,1,0,0,0]: Vertex P has an edge with vertex Q
#[1,1,0,0,1,0]: Vertex Q has an edge with vertex P,O and S
#[1,0,0,0,0,0]: Vertex R has an edge with vertex O
#[0,0,1,0,0,0]: Vertex S has an edge with vertex Q
#[0,0,0,0,0,1]: Vertex T has an edge with itself only since it is the only vertex that speaks the languages E and F
adj_matrix = np.array([[1,0,1,1,0,0],[0,0,1,0,0,0],[1,1,0,0,1,0],[1,0,0,0,0,0],[0,0,1,0,0,0],[0,0,0,0,0,1]])

#this array represents edges*vertices (E*A): e[i][j]==1 if edge i contains vertex j. An edge is a subset of vertices that speak a certain language
#edges considered in this order: {A:{O},B:{O,R},D:{O,Q},C:{Q,P},E:{T},F:{T},G:{S,Q}} - Edge E and F are merged to one edge
e = np.array([[1,0,0,0,0,0],
              [1,0,0,1,0,0],
              [1,0,1,0,0,0],
              [0,1,1,0,0,0],
              [0,0,0,0,0,1],
              [0,0,1,0,1,0]])

#solution using adjacency matrix
# constraints = []
# for i in range(len(adj_matrix)):
#     for j in range(len(adj_matrix[0])):
#         #if there is an edge between node i and node j, either node i or node j needs to be in the vertex cover. The reason is that all edges must be covered
#         if adj_matrix[i][j]==1:
#             constraints.append(X[i]+X[j]>=1)

#solution using e
constraints = [cp.matmul(e,X)>=1]
#constraints = [cp.matmul(a,E)>=1]
for i in range(6):
    constraints.append(X[i]>=0)
problem = cp.Problem(objective, constraints)
result = problem.solve()

In [23]:
result

5.0

The solution says that selecting the vertices O,Q and T provide the optimal vertex cover. I.e. the vertex cover with minimal cost.

In [24]:
X.value

array([1., 0., 1., 0., 0., 1.])

We want to confirm that this indeed is a vertex cover.  
Translator O speaks the languages: ABD.  
Translator Q speaks the languages: CDG.  
Translator T speaks the languages: EF  
So the languages spoken are: ABCDEFG, i.e. all languages are covered by the 3 translators. The cost is 5.

**c)**

In [25]:
X = cp.Variable(6,nonneg=True)
c = np.array([3,1,1,2,4,1])
objective = cp.Minimize(cp.sum(cp.multiply(c,X)))

#Question: Solve using A*E matrix? Solve using adjacency matrix?
#this array represents edges*vertices. e[i][j]==1 if edge i contains vertex j
#the vertices are O,P,Q,R,S,T
e = np.array([[1,0,0,0,0,0],
              [1,0,0,1,0,0],
              [1,0,1,0,0,0],
              [0,1,1,0,0,0],
              [0,0,0,0,0,1],
              [0,0,1,0,1,0]])
constraints = [cp.matmul(e,X)>=1]
problem = cp.Problem(objective, constraints)
result = problem.solve()

In [26]:
result

5.000000000140724

As we can see, the solution using LP relaxation is the same as the solution using IP. The reason is that the optimal solution is the solution in which all coordinates take on Integer Values. The LP relaxation, which uses a larger possible sets of values for the x coordinates, finds the same optimal solution. The LP relaxation allows the coordinates to take on any values, but this does not prevent them from taking on integer values. So if the LP relaxation returns a solution with Integer values, this means that it is the optimal solution globally, for all values of x.

In [27]:
X.value

array([1.00000000e+00, 7.26962845e-11, 1.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.00000000e+00])

# Question 4. 
Consider the ILP and its LP relaxation corresponding to the **VC problem** for the graph $G$ given in the data file. This is a ***random graph*** $G(n,p)$ with $n=100$ vertices generated as follows: for each pair of vertices **independently**, we add an edge with probability $p=0.1$ (so the graph has about 1000 edges).

* **a**. (2 points) Find the optimal solution using **CVXPY**'s integer solver.
* **b**. (2 points) Solve the LP relaxation using **CVXPY** and apply the rounding rule discussed in class to obtain a vertex cover. Compare it to the optimal solution in part (a).
* **c**. (6 points) Consider the following rounding rule: we build up the vertex cover incrementally starting with $S:= \emptyset$. Now consider the edges in $G$ in any order. If an edge $(u,v)$ is already covered by a vertex in $S$, do nothing. Otherwise add to $S$ the vertex $u$ if $x^*(u) \geq x^*(v)$, or $v$ otherwise (where ${\bf x}^*$ is the LP optimum solution computed in part (b).  Comment why this also results in a vertex cover and has cost no more than that corresponding to the rounding rule in part (b). Compare the cost of the solution produced by this rule to the optimal solution.

**a)**

In [27]:
adj_matrix = np.loadtxt("graph.txt")
X_IP = cp.Variable(100, integer=True)
constraints = []
for i in range(100):
    constraints.append(X_IP[i]>=0)
#iterate rows
for u in range(len(adj_matrix)):
    #iterate columns
    for v in range(len(adj_matrix[0])):
        if (adj_matrix[u][v]==1):
            constraints.append(X_IP[u]+X_IP[v]>=1)
objective = cp.Minimize(cp.sum(X_IP))
problem = cp.Problem(objective, constraints)
result_ip = problem.solve()

The result from the IP tells us that the smallest vertex cover has 70 nodes.

In [28]:
result_ip

70.0

In [29]:
X_IP.value

array([1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1.,
       1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
       1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1.,
       1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1.])

**b)**

In [28]:
adj_matrix = np.loadtxt("graph.txt")
X_lp = cp.Variable(100, nonneg=True)
constraints = []
#iterate rows
for u in range(len(adj_matrix)):
    #iterate columns
    for v in range(len(adj_matrix[0])):
        if (adj_matrix[u][v]==1):
            constraints.append(X_lp[u]+X_lp[v]>=1)
objective = cp.Minimize(cp.sum(X_lp))
problem = cp.Problem(objective, constraints)
result_lp = problem.solve()

In [29]:
result_lp

50.00000000007645

In [30]:
X_lp.value

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

The solution we get when rounding the solution from the LP relaxation is 100. This is within a constant factor 2 of the optimal solution of the IP which was 70. The reason that it is within a constant factor 2 of the optimal solution of the IP can be explained by some mathematical reasoning.

In [33]:
np.sum(X_lp.value + 0.5)

100.00000000007645

The optimal solution to the LP is a lower bound on the optimal solution to the corresponding IP. The reason is that the values in the LP relaxation can take on the values in the IP as well as other values between 0 and 1. Since we in the worst case round every value to 1, and in the worst case, every value is 0.5, this means that in the worst case every value is doubled. This means that the value of the solution we get from rounding is in the worst case twice the size of the optimal solution of the LP. Since the optimal solution of the LP is less than or equal to the optimal solution of the IP, this means that the value of the solution we get from rounding the solution of the LP, is in the worst case less than or equal to twice the size of the optimal solution of the IP. Hence it is bounded by a factor of 2, i.e. within a factor of 2 of the optimal solution of the IP.

**c)**

The reason this also results in a vertex cover is that for each edge, we select one node that covers that edge(assuming we have not already selected a node that covers that edge). The result is guaranteed to be a vertex cover. The previous method used in b was a bit different. For an edge, it could select both of its vertices to be in the vertex cover if both of the vertices had the value 0.5. This gave the rounding of LP relaxation method an approximation ratio of 2. However, for this method, if both of the vertices have the value 0.5, it will only select one of the vertices to be in the vertex cover. This gives a tighter approximation ratio.

In [31]:
S = np.zeros(100)
E = []
#add all of the edges to E. Does not add a duplicate edge for example (u,v) and (v,u).
for u in range(len(adj_matrix)):
    for v in range(u,len(adj_matrix[0])):
        if adj_matrix[u][v]==1:
            E.append((u,v))
for e in E:
    u = e[0]
    v = e[1]
    #if the edge is already covered by a node in S
    if (S[u]==1 or S[v]==1):
        continue
    if X_lp.value[u]>=X_lp.value[v]:
        S[u]=1
    else:
        S[v]=1
S

array([1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
       1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0.,
       1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.])

In [32]:
sum(S)

73.0

The cost from using the rounding rule presented in **c)** is 73. This is very close to the optimal solution to the IP which was 70. Hence, we can see that this method of rounding the LP relaxation provides a tighter bound to the optimal solution. The question becomes, **how much tighter is this bound?** We did not manage to get a mathematical upper bound for the method presented in 4c except to prove that it is at least as good as the method presented in 4b)